### 5.1.3.3. 表面熱伝達抵抗

説明：グレージングの表面温度と周囲の放射温度から表面熱伝達抵抗を計算する。

### 入力値  
$ \theta_{sur} $ ：グレージングの表面温度 (℃)  
$ \theta_{r} $ ：周囲の放射温度 (℃)  
$ \epsilon_{sur} $ ：表面放射率   
cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）  

### 出力値
$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ \theta_{sur} $・$ \theta_{r} $・$ \epsilon_{sur} $・cnv_typeの組み合わせを1つの変数（タプル）で表し"shc_input"とする。  

In [19]:
shc_input = namedtuple('shc_input','shc_Tmcs shc_Tmcr shc_eps cnv_type')

# shc_Tmcs：グレージングの表面温度（℃）
# shc_Tmcr：周囲の放射温度（℃）
# shc_eps：表面放射率
# cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）

#### 表面熱伝達抵抗の計算  

（参考：JIS A 2103：2014 6. 境界条件）  

表面熱伝達抵抗の計算方法を以下に示す。 

$ \begin{align}R_{sur} = \frac{1}{h_{r,sur}+h_{c,sur}}\end{align}$ ･･････････････････････････････････ (5.1.3.3-a)  

$ \begin{align}h_{r,sur} = \frac{\epsilon_{sur}･\sigma･(T^4_{sur}-T^4_{r})}{T_{sur}-T_{r}}\end{align}$ ････････････････････････････ (5.1.3.3-b)  

ここで、  

$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ h_{r,sur} $ ：表面放射熱伝達率 [W/(m<sup>2</sup>･K)]  
$ h_{c,sur} $ ：表面対流熱伝達率[W/(m<sup>2</sup>･K)]  
$ \epsilon_{sur} $ ：表面放射率  
$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ T_{sur} $ ：グレージングの表面温度（$T_{sur}=\theta_{sur}+273.15$） (K)   
$ T_{r} $ ：周囲の放射温度（$T_{r}=\theta_{r}+273.15$）(K)   

である。  
なお、本仕様書では、放射温度と空気温度は等しいものとして扱う。  
また、表面対流熱伝達率は、季節や室内外条件に応じた以下の値とする（参考：JIS A 2103：2014 表4）。

| 季節 | 室内外 | 表面対流熱伝達率 $h_{c,sur}$ [W/(m<sup>2</sup>･K)] |
| :---: | :---: | ---: |
| 夏期 | 室内 | 2.5 |
|  | 屋外 | 8.0 |
| 冬期 | 室内 | 3.6 |
|  | 屋外 | 20.0 |

In [20]:
def Surface_Resistance(L):

    Tms = L.shc_Tmcs + 273.15
    Tmr = L.shc_Tmcr + 273.15
    sgm = 5.67 * 10.0 ** (-8.0)
    shc_hr = L.shc_eps * sgm * (Tms ** 4 - Tmr ** 4) / (Tms - Tmr)
    
    shc_hc = [2.5, 8.0, 3.6, 20.0]

    return 1.0 / (shc_hr + shc_hc[L.cnv_type])

#### Example

In [21]:
shc_in = [shc_input(16., 20., 0.837, 2),
          shc_input(4., 0., 0.837, 3)]

shc_result = np.zeros(len(shc_in))
for i in range(len(shc_in)):
    shc_result[i] = Surface_Resistance(shc_in[i])

print (shc_result)

[0.12069538 0.04174568]
